In [ ]:
from brian2 import *
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from src.nb_helpers import *

%matplotlib inline
seed(12345)

In [ ]:
PLOT_ONLY_FROM_EQUILIBRIUM=True
BIN_SIZE_FIRING_RATE = 10
USE_SYNAPSE_PROBS = True


index_to_ntype_dict = {
    0: 'CS',
    1: 'CC',
    2: 'SST',
    3: 'PV'
}

equilibrium_t = 5*second

In [ ]:

################################################################################
# Model parameters
################################################################################
### General parameters
duration = 10*second  # Total simulation time
sim_dt = 0.1*ms        # Integrator/sampling step

# N_sst = N_pv = 100 # Number of SST neurons & PV neurons each (inhibitory)
# N_cc = N_cs = 400 # Number of CC neurons & CS neurons each (excitatory)
N_sst = 100 # Number of SST neurons (inhibitory)
N_pv = 100  # Number of PV neurons (inhibitory)
N_cc = 400  # Number of CC neurons (excitatory)
N_cs = 400  # Number of CS neurons (excitatory)
N = [N_cs, N_cc, N_sst, N_pv]

### Neuron parameters
tau_S   = 16*ms  # 
tau_D   =  7*ms  #
tau_SST = 20*ms  #
tau_PV  = 10*ms  #
tau_E   =  5*ms  # Excitatory synaptic time constant
tau_I   = 10*ms  # Inhibitory synaptic time constant

C_S   = 370*pF
C_D   = 170*pF
C_SST = 100*pF
C_PV  = 100*pF

E_l  = -70*mV   # leak reversal potential
E_e  =   0*mV   # Excitatory synaptic reversal potential
E_i  = -80*mV   # Inhibitory synaptic reversal potential

V_t  = -50*mV   # spiking threashold
V_r  = E_l      # reset potential

c_d = 2600 * pA  # back-propagates somatic spikes to to the dendrites
g_s = 1300 * pA  # propagates dendritic regenerative activity to soma
g_d = 1200 * pA  # propagates dendritic regenerative activity to denderites

### Connectivity weight & probabilities
# CS_CS 0, CS_SST 1, CS_PV 2, SST_CS 3, PV_CS 4, CC_CC 5, CC_SST 6, CC_PV 7, SST_CC 8, PV_CC 9, CC_CS 10, SST_PV 11, SST_SST 12, PV_PV 13, PV_SST 14, 
conn_weights = [0.27, 0.05, 1.01, 0.19, 0.32, 0.24, 0.09, 0.48, 0.19, 0.52, 0.19, 0.18, 0.19, 0.47, 0.44] # in nS
# If not `USE_SYNAPSE_PROBS`, the topology is maintained as specified with a probability p=1
if USE_SYNAPSE_PROBS: 
    # CS_CS 0, CS_SST 1, CS_PV 2, SST_CS 3, PV_CS 4, CC_CC 5, CC_SST 6, CC_PV 7, SST_CC 8, PV_CC 9, CC_CS 10, SST_PV 11, SST_SST 12, PV_PV 13, PV_SST 14, 
    conn_probs = [0.16, 0.23, 0.18, 0.52, 0.43, 0.06, 0.26, 0.22, 0.13, 0.38, 0.09, 0.29, 0.1, 0.5, 0.14]
else: 
    conn_probs = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

### External Input
I_sst_amp = 50
I_pv_amp  = 50
I_cc_amp  = 50
I_cs_amp  = 50

I_sst_steady = 0
I_pv_steady  = 0
I_cc_steady  = 0
I_cs_steady  = 0


lambda_cs = lambda_cc = lambda_sst = lambda_pv = 10*Hz

input_steady = [I_cs_steady, I_cc_steady, I_sst_steady, I_pv_steady]
input_amplitudes = [I_cs_amp, I_cc_amp, I_sst_amp, I_pv_amp]

length = np.random.uniform(0, 1, (np.sum(N),))
angle = np.pi * np.random.uniform(0, 2, (np.sum(N),))
a_data = np.sqrt(length) * np.cos(angle)
b_data = np.sqrt(length) * np.sin(angle)

spatial_F = 10 
temporal_F = 50
spatial_phase = 1
tsteps = int(duration / sim_dt)

sst_soma_conn_weights = [0, 0.2, 0.4, 0.6, 0.8, 1]
degrees = [0, 90, 180, 270]

################################################################################

In [ ]:
### Sigmoid function params
E_d = -38*mV  # position control of threshold
D_d =   6*mV  # sharpness control of threshold 

# TODO see how to reference this from equation 
@check_units(x=volt, result=1)
def sigmoid(x):
    return 1/(1+np.exp(-(-x-E_d)/D_d))

In [ ]:
eqs_sst_inh = '''
    dv/dt = ((E_l-v)/tau_SST + I/C_SST) : volt (unless refractory)

    dg_e/dt = -g_e/tau_E : siemens
    dg_i/dt = -g_i/tau_I : siemens
    
    I = g_e*(E_e - v) + g_i*(E_i - v) : amp
'''

eqs_pv_inh = '''
    dv/dt = ((E_l-v)/tau_PV + I/C_PV) : volt (unless refractory)

    dg_e/dt = -g_e/tau_E : siemens
    dg_i/dt = -g_i/tau_I : siemens

    I = g_e*(E_e - v) + g_i*(E_i - v) : amp
'''

eqs_exc = '''
    dv_s/dt = ((E_l-v_s)/tau_S + (g_s*(1/(1+exp(-(v_d-E_d)/D_d))) + I_s)/C_S) : volt (unless refractory)

    dg_es/dt = -g_es/tau_E : siemens
    dg_is/dt = -g_is/tau_I : siemens

    I_s = g_es*(E_e - v_s) + g_is*(E_i - v_s) : amp

    dv_d/dt = ((E_l-v_d)/tau_D + (g_d*(1/(1+exp(-(v_d-E_d)/D_d))) + c_d*K + I_d)/C_D) : volt

    dg_ed/dt = -g_ed/tau_E : siemens
    dg_id/dt = -g_id/tau_I : siemens

    I_d = g_ed*(E_e - v_d) + g_id*(E_i - v_d) : amp
    K : 1
'''

In [ ]:
def compute_and_plot_fire_rates_histogram(spike_monitors):
    spike_mon_sst, spike_mon_pv, spike_mon_cs, spike_mon_cc = spike_monitors
    
    from_t = equilibrium_t if PLOT_ONLY_FROM_EQUILIBRIUM else 0
    to_t = duration

    # Compute firing rate for each neuron group
    firing_rates_cs = compute_firing_rate_for_neuron_type(spike_mon_cs, from_t, to_t)
    firing_rates_cc = compute_firing_rate_for_neuron_type(spike_mon_cc, from_t, to_t)
    firing_rates_sst = compute_firing_rate_for_neuron_type(spike_mon_sst, from_t, to_t)
    firing_rates_pv = compute_firing_rate_for_neuron_type(spike_mon_pv, from_t, to_t)

    firing_rates = [firing_rates_cs, firing_rates_cc, firing_rates_sst, firing_rates_pv]

    columns = 2
    rows = int(len(firing_rates) / columns)

    fig, axs = plt.subplots(rows, columns, figsize = (12, 9))

    for (ntype_index, firing_rate_i) in enumerate(firing_rates):
        row_idx = int(ntype_index / columns)
        col_idx = ntype_index % columns

        axs[row_idx][col_idx].hist(firing_rate_i, bins=BIN_SIZE_FIRING_RATE)
        axs[row_idx][col_idx].axis(ymin=0)
        axs[row_idx][col_idx].set_title(f'Neuron group {index_to_ntype_dict[ntype_index]}', fontsize = 10)
        axs[row_idx][col_idx].set_ylabel("Frequency", fontsize = 10)
        axs[row_idx][col_idx].set_xlabel("Firing rate [Hz]", fontsize = 10)
        axs[row_idx][col_idx].tick_params(axis='both', which='major', labelsize=10)


    plt.tight_layout()
    plt.show()
    
    return firing_rates



def simulate_network(input_degree, sst_soma_conn_weight=0.5):
    start_scope()
    
    rad = math.radians(input_degree)
    inputs = distributionInput(
        a_data=a_data, b_data=b_data,
        spatialF=spatial_F, temporalF=temporal_F, orientation=rad,
        spatialPhase=spatial_phase, amplitude=input_amplitudes, T=tsteps,
        steady_input=input_steady, N=N
    )

    I_ext_cs  = TimedArray(inputs[:, :N_cs]*nS, dt=sim_dt)
    I_ext_cc  = TimedArray(inputs[:, N_cs:N_cs+N_cc]*nS, dt=sim_dt)
    I_ext_sst = TimedArray(inputs[:, N_cs+N_cc:N_cs+N_cc+N_sst]*nS, dt=sim_dt)
    I_ext_pv  = TimedArray(inputs[:, N_cs+N_cc+N_sst:]*nS, dt=sim_dt)

    
    # ##############################################################################
    # Neurons
    # ##############################################################################
    
    # SST Neurons
    sst_neurons = NeuronGroup(N_sst, model=eqs_sst_inh, threshold='v > V_t',
                                  reset='v = E_l', refractory=8.3 * ms, method='euler')
    sst_neurons.v = 'E_l + rand()*(V_t-E_l)'

    ## Poisson input to SST neurons
    for n_idx in range(N_sst):
        sst_input_i = PoissonInput(sst_neurons, 'g_e', N=1, rate=lambda_sst, weight=f'I_ext_sst(t, {n_idx})')

    # PV Neurons
    pv_neurons = NeuronGroup(N_pv, model=eqs_pv_inh, threshold='v > V_t',
                                 reset='v = E_l', refractory=8.3 * ms, method='euler')
    pv_neurons.v = 'E_l + rand()*(V_t-E_l)'

    ## Poisson input to PV neurons
    for n_idx in range(N_pv):
        pv_input_i = PoissonInput(pv_neurons, 'g_e', N=1, rate=lambda_pv, weight=f'I_ext_pv(t, {n_idx})')

    # CS Neurons
    cs_neurons = NeuronGroup(N_cs, model=eqs_exc, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cs_neurons.v_s = 'E_l + rand()*(V_t-E_l)'
    cs_neurons.v_d = -70 * mV

    ## Poisson input to CS neurons
    for n_idx in range(N_cs):
        cs_input_i = PoissonInput(cs_neurons, 'g_es', N=1, rate=lambda_cs, weight=f'I_ext_cs(t, {n_idx})')

    # CC Neurons
    cc_neurons = NeuronGroup(N_cc, model=eqs_exc, threshold='v_s > V_t',
                                 reset='v_s = E_l', refractory=8.3 * ms, method='euler')
    cc_neurons.v_s = 'E_l + rand()*(V_t-E_l)'
    cc_neurons.v_d = -70 * mV

    ## Poisson input to CC neurons
    for n_idx in range(N_cc):
        cc_input_i = PoissonInput(cc_neurons, 'g_es', N=1, rate=lambda_cc, weight=f'I_ext_cc(t, {n_idx})')
        
    # ##############################################################################
    # Synapses
    # ##############################################################################
    
    # SST <=> PV
    SST_PV = 11
    conn_SST_PV = Synapses(sst_neurons, pv_neurons, model='w: 1', on_pre='g_i+=w*nS', name='SST_PV') # inhibitory
    conn_SST_PV.connect(p=conn_probs[SST_PV])
    conn_SST_PV.w = conn_weights[SST_PV]

    PV_SST = 14
    conn_PV_SST = Synapses(pv_neurons, sst_neurons, model='w: 1', on_pre='g_i+=w*nS', name='PV_SST') # inhibitory
    conn_PV_SST.connect(p=conn_probs[PV_SST])
    conn_PV_SST.w = conn_weights[PV_SST]

    # PV <=> PYR soma
    ## target CS soma
    PV_CSsoma = 4
    conn_PV_CSsoma = Synapses(pv_neurons, cs_neurons, model='w: 1', on_pre='g_is+=w*nS', name='PV_CSsoma') # inhibitory
    conn_PV_CSsoma.connect(p=conn_probs[PV_CSsoma])
    conn_PV_CSsoma.w = conn_weights[PV_CSsoma]

    CSsoma_PV = 2
    conn_CSsoma_PV = Synapses(cs_neurons, pv_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CSsoma_PV') # excitatory
    conn_CSsoma_PV.connect(p=conn_probs[CSsoma_PV])
    conn_CSsoma_PV.w = conn_weights[CSsoma_PV]

    ## target CC soma
    PV_CCsoma = 9
    conn_PV_CCsoma = Synapses(pv_neurons, cc_neurons, model='w: 1', on_pre='g_is+=w*nS', name='PV_CCsoma') # inhibitory 
    conn_PV_CCsoma.connect(p=conn_probs[PV_CCsoma])
    conn_PV_CCsoma.w = conn_weights[PV_CCsoma]


    CCsoma_PV = 7
    conn_CCsoma_PV = Synapses(cc_neurons, pv_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CCsoma_PV') # excitatory
    conn_CCsoma_PV.connect(p=conn_probs[CCsoma_PV])
    conn_CCsoma_PV.w = conn_weights[CCsoma_PV]
    
    # SST => PYR dendrite
    ## target CS dendrite
    SST_CSdendrite = 3
    conn_SST_CSdendrite = Synapses(sst_neurons, cs_neurons, model='w: 1', on_pre='g_id+=w*nS', name='SST_CSdendrite') # inhibitory
    conn_SST_CSdendrite.connect(p=conn_probs[SST_CSdendrite] * (1-sst_soma_conn_weight)) # prob divided between soma & dendrite
    conn_SST_CSdendrite.w = conn_weights[SST_CSdendrite]

    ## target CC dendrite
    SST_CCdendrite = 8
    conn_SST_CCdendrite = Synapses(sst_neurons, cc_neurons, model='w: 1', on_pre='g_id+=w*nS', name='SST_CCdendrite') # inhibitory
    conn_SST_CCdendrite.connect(p=conn_probs[SST_CCdendrite] * (1-sst_soma_conn_weight)) # prob divided between soma & dendrite
    conn_SST_CCdendrite.w = conn_weights[SST_CCdendrite]

    # SST <=> PYR soma
    ## target CS soma
    SST_CSsoma = 3
    conn_SST_CSsoma = Synapses(sst_neurons, cs_neurons, model='w: 1', on_pre='g_is+=w*nS', name='SST_CSsoma') # inhibitory (optional connection)
    conn_SST_CSsoma.connect(p=conn_probs[SST_CSsoma] * sst_soma_conn_weight) # prob divided between soma & dendrite
    conn_SST_CSsoma.w = conn_weights[SST_CSsoma]

    CSsoma_SST = 1
    conn_CSsoma_SST = Synapses(cs_neurons, sst_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CSsoma_SST') # excitatory
    conn_CSsoma_SST.connect(p=conn_probs[CSsoma_SST])
    conn_CSsoma_SST.w = conn_weights[CSsoma_SST]

    ## taget CC soma
    SST_CCsoma = 8
    conn_SST_CCsoma = Synapses(sst_neurons, cc_neurons, model='w: 1', on_pre='g_is+=w*nS', name='SST_CCsoma') # inhibitory (optional connection)
    conn_SST_CCsoma.connect(p=conn_probs[SST_CCsoma] * sst_soma_conn_weight) # prob divided between soma & dendrite
    conn_SST_CCsoma.w = conn_weights[SST_CCsoma]

    CCsoma_SST = 6
    conn_CCsoma_SST = Synapses(cc_neurons, sst_neurons, model='w: 1', on_pre='g_e+=w*nS', name='CCsoma_SST') # excitatory
    conn_CCsoma_SST.connect(p=conn_probs[CCsoma_SST])
    conn_CCsoma_SST.w = conn_weights[CCsoma_SST]

    # CC => CS 
    ## target CS soma
    CCsoma_CSsoma = 10
    conn_CCsoma_CSsoma = Synapses(cc_neurons, cs_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CC_CSsoma') # excitatory
    conn_CCsoma_CSsoma.connect(p=conn_probs[CCsoma_CSsoma])
    conn_CCsoma_CSsoma.w = conn_weights[CCsoma_CSsoma]

    # self connections
    ## CS soma self connection
    CSsoma_CSsoma = 0
    conn_CSsoma_CSsoma = Synapses(cs_neurons, cs_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CSsoma_CSsoma')  # excitatory
    conn_CSsoma_CSsoma.connect(p=conn_probs[CSsoma_CSsoma])
    conn_CSsoma_CSsoma.w = conn_weights[CSsoma_CSsoma]

    backprop_CS = Synapses(cs_neurons, cs_neurons, on_pre={'up': 'K += 1', 'down': 'K -=1'},
                               delay={'up': 0.5 * ms, 'down': 2 * ms}, name='backprop_CS')
    backprop_CS.connect(condition='i==j')  # Connect all CS neurons to themselves

    ## CC soma self connection
    CCsoma_CCsoma = 5
    conn_CCsoma_CCsoma = Synapses(cc_neurons, cc_neurons, model='w: 1', on_pre='g_es+=w*nS', name='CCsoma_CCsoma')  # excitatory
    conn_CCsoma_CCsoma.connect(p=conn_probs[CCsoma_CCsoma])
    conn_CCsoma_CCsoma.w = conn_weights[CCsoma_CCsoma]

    backprop_CC = Synapses(cc_neurons, cc_neurons, on_pre={'up': 'K += 1', 'down': 'K -=1'},
                               delay={'up': 0.5 * ms, 'down': 2 * ms}, name='backprop_CC')
    backprop_CC.connect(condition='i==j')  # Connect all CC neurons to themselves

    ## SST self connection
    SST_SST = 12
    conn_SST_SST = Synapses(sst_neurons, sst_neurons, model='w: 1', on_pre='g_i+=w*nS', name='SST_SST')  # inhibitory
    conn_SST_SST.connect(p=conn_probs[SST_SST])
    conn_SST_SST.w = conn_weights[SST_SST]

    ## PV self connection
    PV_PV = 13
    conn_PV_PV = Synapses(pv_neurons, pv_neurons, model='w: 1', on_pre='g_i+=w*nS', name='PV_PV')  # inhibitory
    conn_PV_PV.connect(p=conn_probs[PV_PV])
    conn_PV_PV.w = conn_weights[PV_PV]
    
    # ##############################################################################
    # Monitors
    # ##############################################################################
    
    # Record spikes of different neuron groups
    spike_mon_sst = SpikeMonitor(sst_neurons)
    spike_mon_pv = SpikeMonitor(pv_neurons)
    spike_mon_cs = SpikeMonitor(cs_neurons)
    spike_mon_cc = SpikeMonitor(cc_neurons)
    
    spike_monitors = [spike_mon_sst, spike_mon_pv, spike_mon_cs, spike_mon_cc]
    
    network = Network(collect())
    
    print(f"Running simulations for input of degree {input_degree} ... WITH SST->SOMA*{sst_soma_conn_weight} & SST->DENDRITE*{1-sst_soma_conn_weight} ...")
    
    defaultclock.dt = sim_dt
    network.run(duration, report='text')
    
    firing_rates = compute_and_plot_fire_rates_histogram(spike_monitors)
    
    return firing_rates


In [ ]:
simulation_firing_rates = []

In [ ]:
# ##############################################################################
# # Simulation run for degree 0
# ##############################################################################

# Simulate for each `sst_soma_conn_weights`
firing_rates_0 = []
for sst_soma_weight in sst_soma_conn_weights:
    firing_rates = simulate_network(input_degree=0, sst_soma_conn_weight=sst_soma_weight) 
    firing_rates_cs_0, firing_rates_cc_0, firing_rates_sst_0, firing_rates_pv_0 = firing_rates
    firing_rates_0.append([np.mean(rate) for rate in firing_rates])
    
    print(f'Avg firing rate for CS neurons: {np.mean(firing_rates_cs_0) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for CC neurons: {np.mean(firing_rates_cc_0) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for SST neurons: {np.mean(firing_rates_sst_0) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for PV neurons: {np.mean(firing_rates_pv_0) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print('\n============================================================================================')
    

simulation_firing_rates.append(firing_rates_0) # Append `firing_rates_0` to the overall simulation result vector for inputs


In [ ]:
# ##############################################################################
# # Simulation run for degree 90
# ##############################################################################

# Simulate for each `sst_soma_conn_weights`
firing_rates_90 = []
for sst_soma_weight in sst_soma_conn_weights:
    firing_rates = simulate_network(input_degree=90, sst_soma_conn_weight=sst_soma_weight) 
    firing_rates_cs_90, firing_rates_cc_90, firing_rates_sst_90, firing_rates_pv_90 = firing_rates
    firing_rates_90.append([np.mean(rate) for rate in firing_rates])
    
    print(f'Avg firing rate for CS neurons: {np.mean(firing_rates_cs_90) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for CC neurons: {np.mean(firing_rates_cc_90) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for SST neurons: {np.mean(firing_rates_sst_90) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for PV neurons: {np.mean(firing_rates_pv_90) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print('\n============================================================================================')
    

simulation_firing_rates.append(firing_rates_90) # Append `firing_rates_90` to the overall simulation result vector for inputs


In [ ]:
# ##############################################################################
# # Simulation run for degree 180
# ##############################################################################

# Simulate for each `sst_soma_conn_weights`
firing_rates_180 = []
for sst_soma_weight in sst_soma_conn_weights:
    firing_rates = simulate_network(input_degree=180, sst_soma_conn_weight=sst_soma_weight) 
    firing_rates_cs_180, firing_rates_cc_180, firing_rates_sst_180, firing_rates_pv_180 = firing_rates
    firing_rates_180.append([np.mean(rate) for rate in firing_rates])
    
    print(f'Avg firing rate for CS neurons: {np.mean(firing_rates_cs_180) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for CC neurons: {np.mean(firing_rates_cc_180) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for SST neurons: {np.mean(firing_rates_sst_180) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for PV neurons: {np.mean(firing_rates_pv_180) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print('\n============================================================================================')
    

simulation_firing_rates.append(firing_rates_180) # Append `firing_rates_180` to the overall simulation result vector for inputs


In [ ]:
# ##############################################################################
# # Simulation run for degree 270
# ##############################################################################

# Simulate for each `sst_soma_conn_weights`
firing_rates_270 = []
for sst_soma_weight in sst_soma_conn_weights:
    firing_rates = simulate_network(input_degree=270, sst_soma_conn_weight=sst_soma_weight) 
    firing_rates_cs_270, firing_rates_cc_270, firing_rates_sst_270, firing_rates_pv_270 = firing_rates
    firing_rates_270.append([np.mean(rate) for rate in firing_rates])
    
    print(f'Avg firing rate for CS neurons: {np.mean(firing_rates_cs_270) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for CC neurons: {np.mean(firing_rates_cc_270) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for SST neurons: {np.mean(firing_rates_sst_270) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print(f'Avg firing rate for PV neurons: {np.mean(firing_rates_pv_270) * Hz} [SST->SOMA*{sst_soma_weight}, SST->DENDRITE*{1-sst_soma_weight}]')
    print('\n============================================================================================')
    

simulation_firing_rates.append(firing_rates_270) # Append `firing_rates_270` to the overall simulation result vector for inputs


In [ ]:
rows = len(sst_soma_conn_weights)
fig, axs = plt.subplots(rows, 3, figsize=(26, rows*6))

for row_idx, sst_soma_weight in enumerate(sst_soma_conn_weights):
    bar_width = 0.4
    x = np.arange(1) / 2
    labels = [f'SST->Soma*{sst_soma_weight}']
    
    # compute selectivities for simulation
    selectivity_CS = calculate_selectivity([sim_rate[row_idx][0] for sim_rate in simulation_firing_rates])
    selectivity_CC = calculate_selectivity([sim_rate[row_idx][1] for sim_rate in simulation_firing_rates])
    selectivity_SST = calculate_selectivity([sim_rate[row_idx][2] for sim_rate in simulation_firing_rates])
    selectivity_PV = calculate_selectivity([sim_rate[row_idx][3] for sim_rate in simulation_firing_rates])

    os_cs = [selectivity_CS["orientation"]]
    os_cc = [selectivity_CC["orientation"]]
    os_sst = [selectivity_SST["orientation"]]
    os_pv = [selectivity_PV["orientation"]]

    # plot orientation selectivity
    axs[row_idx][0].bar(x, os_cs, bar_width / 4, label="CS", color='b')
    axs[row_idx][0].bar(x + bar_width / 4, os_cc, bar_width / 4, label="CC", color='r')
    axs[row_idx][0].bar(x + bar_width / 2, os_sst, bar_width / 4, label="SST", color='g')
    axs[row_idx][0].bar(x + bar_width * 3 / 4, os_pv, bar_width / 4, label="PV", color='y')
    axs[row_idx][0].set_ylabel('Orientation selectivity')
    axs[row_idx][0].set_title('Orientation selectivity')
    axs[row_idx][0].set_xticks(x + bar_width / 4)
    axs[row_idx][0].set_xticklabels(labels)
    axs[row_idx][0].legend(loc='best')

    os_paper_cs = [selectivity_CS["orientation_paper"]]
    os_paper_cc = [selectivity_CC["orientation_paper"]]
    os_paper_sst = [selectivity_SST["orientation_paper"]]
    os_paper_pv = [selectivity_PV["orientation_paper"]]

    # plot orientation selectivity
    axs[row_idx][1].bar(x, os_paper_cs, bar_width / 4, label="CS", color='b')
    axs[row_idx][1].bar(x + bar_width / 4, os_paper_cc, bar_width / 4, label="CC", color='r')
    axs[row_idx][1].bar(x + bar_width / 2, os_paper_sst, bar_width / 4, label="SST", color='g')
    axs[row_idx][1].bar(x + bar_width * 3 / 4, os_paper_pv, bar_width / 4, label="PV", color='y')
    axs[row_idx][1].set_ylabel('Orientation selectivity (paper)')
    axs[row_idx][1].set_title('Orientation selectivity (paper)')
    axs[row_idx][1].set_xticks(x + bar_width / 4)
    axs[row_idx][1].set_xticklabels(labels)
    axs[row_idx][1].legend(loc='best')

    ds_cs = [selectivity_CS["direction"]]
    ds_cc = [selectivity_CC["direction"]]
    ds_sst = [selectivity_SST["direction"]]
    ds_pv = [selectivity_PV["direction"]]

    # plot direction selectivity
    axs[row_idx][2].bar(x, ds_cs, bar_width / 4, label="CS", color='b')
    axs[row_idx][2].bar(x + bar_width / 4, ds_cc, bar_width / 4, label="CC", color='r')
    axs[row_idx][2].bar(x + bar_width / 2, ds_sst, bar_width / 4, label="SST", color='g')
    axs[row_idx][2].bar(x + bar_width * 3 / 4, ds_pv, bar_width / 4, label="PV", color='y')
    axs[row_idx][2].set_ylabel('Direction selectivity')
    axs[row_idx][2].set_title('Direction selectivity')
    axs[row_idx][2].set_xticks(x + bar_width / 4)
    axs[row_idx][2].set_xticklabels(labels)
    axs[row_idx][2].legend(loc='best')